In [ ]:
# To run Jupyterlab on lab computers, please use 'python -m jupyterlab' in the Anaconda Prompt

In [ ]:
%pylab inline    # This is useful. It basically imports numpy and matplotlib. So you don't need to add 'np.' or 'plt.' to the front of the functions

# Comminication with devices

## NI USB-6008 to generate the voltage to control the nanopositioning system

In [ ]:
import PyDAQmx as pydaqmx

In [ ]:
'''
Output 2.5 V on ao0 channel of Dev2
The address '/Dev2/ao0' can be found in NI MAX and the channel list of USB-6008 device
'''

z_ao = 2.5

task_z = pydaqmx.Task()
task_z.CreateAOVoltageChan("/Dev2/ao0","", 0,5.0,pydaqmx.DAQmx_Val_Volts,None)

task_z.StartTask()
task_z.WriteAnalogScalarF64(1,10.0,z_ao,None)
task_z.StopTask()


## Another option: use MCC USB 3101

In [1]:
from mcculw import ul
from mcculw.enums import InterfaceType

In [7]:
def config_first_detected_device(board_num, dev_id_list=None):
    """Adds the first available device to the UL.  If a types_list is specified,
    the first available device in the types list will be add to the UL.

    Parameters
    ----------
    board_num : int
        The board number to assign to the board when configuring the device.

    dev_id_list : list[int], optional
        A list of product IDs used to filter the results. Default is None.
        See UL documentation for device IDs.
    """
    ul.ignore_instacal()
    devices = ul.get_daq_device_inventory(InterfaceType.ANY)
    if not devices:
        raise Exception('Error: No DAQ devices found')

    print('Found', len(devices), 'DAQ device(s):')
    for device in devices:
        print('  ', device.product_name, ' (', device.unique_id, ') - ',
              'Device ID = ', device.product_id, sep='')

    device = devices[0]
    if dev_id_list:
        device = next((device for device in devices
                       if device.product_id in dev_id_list), None)
        if not device:
            err_str = 'Error: No DAQ device found in device ID list: '
            err_str += ','.join(str(dev_id) for dev_id in dev_id_list)
            raise Exception(err_str)

    # Add the first DAQ device to the UL with the specified board number
    ul.create_daq_device(board_num, device)

In [10]:
board_num=0
config_first_detected_device(board_num)

Found 1 DAQ device(s):
  USB-3101 (2065A61) - Device ID = 154


In [ ]:
def voltage_to_data(volt):
    return int(volt/10*(2**16-1))

In [12]:
'''
Analog output on channel 0
This device has 16-bit resolution. It accepts 'data_value' from 0~65535(or 2**16-1) corresponding to 0~10 V
''' 
ul.a_out(board_num, channel=0, ul_range=0, data_value=voltage_to_data(5))

## Swabian Time Tagger

In [ ]:
import TimeTagger

In [ ]:
# Counter
tagger = TimeTagger.createTimeTagger()
binwidth = 0.1e12    # unit: ps
n_values = 1    # number of points to record for each readout
counter = TimeTagger.Counter(tagger=tagger, channels=[1], binwidth=binwidth, n_values=n_values)

counter.getData()    # This will give you a 2D array: (number of channels, n_values)

In [ ]:
# Free the TimeTagger at the end of your experiment. Otherwise you won't be able to use it in other codes or the Web Application
TimeTagger.freeTimeTagger(tagger)

In [ ]:
'''
Example: get count rate from the TimeTagger
'''
def get_count_rate(binwidth=0.1e12):
    tagger = TimeTagger.createTimeTagger()

    n_values = 1
    counter = TimeTagger.Counter(tagger=tagger, channels=[1], binwidth=binwidth, n_values=n_values)
    time.sleep(binwidth*n_values)
    
    PL = counter.getData()[0][0]/(binwidth/1e12)
    TimeTagger.freeTimeTagger(tagger)
    return PL

## Digilent Analog Discovery 2 

In [ ]:
from ctypes import *
from dwfconstants import *    # you can copy this file from the Digilent example code folder

In [ ]:
# Initialization

dwf = cdll.dwf
hdwf = c_int()
version = create_string_buffer(16)
dwf.FDwfGetVersion(version)
print("DWF Version: "+str(version.value))

print("Opening first device")
hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))

if hdwf.value == 0:
    print("failed to open device")
    szerr = create_string_buffer(512)
    dwf.FDwfGetLastErrorMsg(szerr)
    print(str(szerr.value))
    quit()

In [ ]:
'''
This sets the 'V+' and 'V-' channel voltages to +5 V and -5V respectively, which is required by the microwave switch.
'''

# set up analog IO channel nodes
# enable positive supply
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(True)) 
# set voltage to 5 V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(5.0)) 
# enable negative supply
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(1), c_int(0), c_double(True)) 
# set voltage to -5 V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(1), c_int(1), c_double(-5.0)) 

dwf.FDwfAnalogIOEnableSet(hdwf, c_int(True))

In [ ]:
'''
Configure the custom digital output for channel 'idxChannel'. 
The output pattern can be designed by an array 'data'. Each entry in the array lasts for 10 ns.
'''

def Configure_DO_channel(device_handle, idxChannel, data):
    # enable the respective channel
    dwf.FDwfDigitalOutEnableSet(device_handle, c_int(idxChannel), c_int(1))
    # set output type
    dwf.FDwfDigitalOutTypeSet(device_handle, c_int(idxChannel), DwfDigitalOutTypeCustom)
    
    # format data
    # how many bytes we need to fit this many bits, (+7)/8
    buffer = (c_ubyte * ((len(data) + 7) >> 3))(0)
    # array to bits in byte array
    for index in range(len(data)):
        if data[index] != 0:
            buffer[index >> 3] |= 1 << (index & 7)

    # load data
    dwf.FDwfDigitalOutDataSet(device_handle, c_int(idxChannel), byref(buffer), c_int(len(data)))

In [ ]:
# turn on Digital output
dwf.FDwfDigitalOutConfigure(hdwf, c_int(1))

# turn off Digital output
dwf.FDwfDigitalOutConfigure(hdwf, c_int(0))

In [ ]:
'''
Example: turn on DO0 to turn on green laser
'''

Configure_DO_channel(hdwf, 0, [1])

# turn on Digital output
dwf.FDwfDigitalOutConfigure(hdwf, c_int(1))

In [ ]:
# Close the connection to the device at the end of your experiment.
dwf.FDwfDigitalOutReset(hdwf)
dwf.FDwfDeviceCloseAll()

## SRS SG384 signal generator

In [ ]:
import pyvisa as visa

In [ ]:
# This part is from the paper: Nature Protocols 14.9 (2019): 2707-2747.

def SRSerrCheck(SRS):
    err = SRS.query('LERR?')
    if int(err) is not 0:
        print('SRS error: error code', int(err),'. Please refer to SRS manual for a description of error codes.')
        sys.exit()
    
def enableSRS_RFOutput(SRS):
    SRS.write('ENBR 1')
    SRSerrCheck(SRS)

def disableSRS_RFOutput(SRS):
    SRS.write('ENBR 0')
    SRSerrCheck(SRS)
    
def setSRS_RFAmplitude(SRS,RFamplitude, units='dBm'):
    SRS.write('AMPR '+str(RFamplitude)+' '+units)
    SRSerrCheck(SRS)

def setSRS_Freq(SRS,freq, units='Hz'):
    #setSRSFreq: Sets frequency of the SRS output. You can call this function with one argument only (the first argument, freq),
    # in which case the argument freq must be in Hertz. This function can also be called with both arguments, the first
    # specifying the frequency and the second one specifying the units, as detailed below.
    # arguments: - freq: float setting frequency of SRS. This must either be in Hz if the units argument is not passed.
    #            - units: string describing units (e.g. 'MHz'). For SRS384, minimum unit is 'Hz', max 'GHz'
    SRS.write('FREQ '+str(freq)+' '+units)
    SRSerrCheck(SRS)
    
def enableIQmodulation(SRS):
    SRSerrCheck(SRS)
    #Enable modulation
    SRS.write('MODL 1')
    SRSerrCheck(SRS)
    #Set modulation type to IQ
    SRS.write('TYPE 6')
    SRSerrCheck(SRS)
    #Set IQ modulation function to external
    SRS.write('QFNC 5')
    
def disableModulation(SRS):
    SRS.write('MODL 0')
    SRSerrCheck(SRS)
    
def queryModulationStatus(SRS):
    status = SRS.query('MODL?')
    SRSerrCheck(SRS)
    if status=='1\r\n':
        print('SRS modulation is on...')
        IQstatus = SRS.query('TYPE?')
        SRSerrCheck(SRS)
        if IQstatus=='6\r\n':
            print('...and is set to IQ')
        else:
            print('...but is not set to IQ.')
    else:
        print('SRS modulation is off.')
    return status

In [ ]:
# Connect with the device
rm = visa.ResourceManager()
SRS = rm.open_resource('TCPIP0::169.254.209.144::inst0::INSTR')    # you can find the address from NI MAX

In [ ]:
# set the MW frequency, unit: HZ
setSRS_Freq(SRS, 2.87e6)    

# Set the MW amplitude, unit: dBm
setSRS_RFAmplitude(SRS, 0)

## Keysight 33622A waveform generator

In [ ]:
class keysight33622A():
    def __init__(self, address):
        self.connected = False
        rm = visa.ResourceManager()
        
        try:
            self.inst = rm.open_resource(address, access_mode=4) # access_mode set to load interface params from NI-Max
            self.connected = True
        except:
            warnings.warn('Dev %s not found.' % address)
    def set_timeout(self, timeout):
        self.inst.timeout = timeout
    
    def set_func(self, ch, fun):
        self.inst.write('SOUR%d:FUNC %s' % (ch, fun))

    def set_freq(self, ch, freq):
        self.inst.write('SOUR%d:FREQ %.3f' % (ch, freq))

    def set_amplitude(self, ch, amp):
        self.set_limits()
        self.inst.write('SOUR%d:VOLT %.3f' % (ch, amp))

    def set_dc(self, ch, v):
        self.inst.write('SOUR%d:VOLT:OFFS %.3f' % (ch, v))

    def set_amplitude_wfm(self, ch, low, high):
        if low == 0 and high == 0:
            self.inst.write('SOUR%d:VOLT:LOW MIN' % (ch))
            self.inst.write('SOUR%d:VOLT:HIGH MAX' % (ch))
        else:
            self.inst.write('SOUR%d:VOLT:LOW %.3f' % (ch, low))
            self.inst.write('SOUR%d:VOLT:HIGH %.3f' % (ch, high))

    def set_output(self, b):
        self.inst.write('OUTP1 %d' % b)
        self.inst.write('OUTP2 %d' % b)

    def set_triggered(self, ch, b):
        self.inst.write('SOUR%d:BURS:STAT %d' % (ch, b))
        if bool(b):
            self.inst.write('SOUR%d:BURS:MODE TRIG' % ch)
            self.inst.write('SOUR%d:BURS:NCYC 1' % ch)
            self.inst.write('TRIG%d:SOUR EXT' % ch)

    def set_wfm(self, ch, wfm, sampl=250e6):
        self.set_func(ch, 'ARB') # arbitrary function
        self.inst.write('SOUR%d:FUNC:ARB:SRAT %d' % (ch, sampl)) # sample rate
        self.inst.write('SOUR%d:FUNC:ARB:FILT OFF' % ch) # Arbitrary filter
        self.set_triggered(ch, 1)

        self.inst.write('SOUR%d:DATA:VOL:CLE' % ch) # Data volatile clear

        wfm_array = np.array(wfm)

        header = 'SOUR%d:DATA:ARB wfm%d, ' % (ch, ch)

        self.inst.write('FORMat:BORDer SWAP')

        # this function takes care of the binary block header by itself
        self.inst.write_binary_values(header, wfm_array)

        self.inst.write('SOUR%d:FUNC:ARB wfm%d' % (ch, ch))

    def set_wfm_dual(self, wfm1, wfm2, sampl=250e6):
        # wfm is a list of float, normalized to 1
        self.set_wfm(1, wfm1, sampl)
        self.set_wfm(2, wfm2, sampl)

    def set_limits(self):
        self.inst.write('SOUR1:VOLT:LIM:STAT 1') # voltage limit state 0:off, 1:on
        self.inst.write('SOUR1:VOLT:LIM:LOW %f' % -0.5)
        self.inst.write('SOUR1:VOLT:LIM:HIGH %f' % 0.5)
        self.inst.write('SOUR2:VOLT:LIM:STAT 1')
        self.inst.write('SOUR2:VOLT:LIM:LOW %f' % -0.5)
        self.inst.write('SOUR2:VOLT:LIM:HIGH %f' % 0.5)

    def get_error(self):
        err = self.inst.query('SYST:ERR?')
        
        if '+0' in err:
            print(err)
        else:
            error_all = ''
            max_err = 20  # maximum number of errors to read - prevent infinite loop
            itr = 0
            while '+0' not in err and itr < max_err:
                error_all += err
                err = self.inst.query('SYST:ERR?')
                itr += 1
            if itr >= max_err:
                print('More than %d error messages occurred. You are probably doing something stupid...' % max_err)
            return error_all

    def set_view(self, mode):
        # mode: STANdard|TEXT|GRAPh|DUAL
        self.inst.write('DISP:VIEW %s' % mode)
    
    def beep(self):
        self.inst.write('SYST:BEEP')
    def set_beep(self, b):
#         awg can still beep by running self.beep(), but it won't beep when error occurs
        self.inst.write('SYST:BEEP:STAT %d' % b)

In [ ]:
awg = keysight33622A(address='AWG_deLeon_lab')    # you can find the address in NI MAX

# Confocal Microscopy

## Turn on the green laser
- Turn on Digilent Digital Ouput channel 0

## Focus the laser on the diamond surface
- Use the manual translation stage and check through the ThorCam to find the smallest laser spot position

## Move the nanopositioning system to focus at a few microns inside the diamond
- Scan the nanopositioning system in Z direction and get the counts from TimeTagger for each step.
- Adjust the binwidth of the TimeTagger until you get reasonable sigal-to-noise ratio.
- Make a plot and fit with a proper function to find the diamond surface position.
- Move the Z position to look inside the diamond. How can you tell which direction the nanopositioning system is moving in?

In [ ]:
# Hint: when you move the nanopositioning system or collect data from TimeTagger. Allow the devices enough time to finish.
time.sleep(1)    # add sleep time in your code, unit: second

## Scan the nanopositioning system in X and Y directions and get a confocal scan image
- Estimate the binwidth you need based on the count rate of an NV center.
- Estimate the size and shape of an NV center you can see from a confocal image. Use this to find the resolution you need in your confocal scan and check whether what you find is an NV.
- You might see signal from other sources. Move to a region with low background counts to find your NV center. 
- If you can't find any NV, try scanning another region or moving to another Z position.

## Find the accurate positions of NV centers
- Run X, Y and Z scans around the rough NV positions you find from your confocal image. Fit your data to get the accurate positions.
- Store your NV positions and confocal image properly so that you can go back to your NVs later.
- Move the nanopositioning system to one NV center for following experiments.

# ODMR 

## Power on the microwave amplifier
- Please power off the high power amplifier when you leave the lab.

## Sample drifting by microwave
- When you turn on the microwave signal, the sample will drift relative to the objective. You will notice a decrease in your PL signal. 
- Write a function to easily run the X, Y and Z scans to update the NV positions. You will need to run this frequently to keep track of your NV center.
- To minimize the drift, keep microwave power less than 0 dBm. Also design the experiment to decrease microwave duty cycle. 


## Signal v.s. Reference
- As you will see a decrease in your overall PL signal, it'll be better to collect some reference signal to cancel out this overall drift. You can create two counters from the TimeTagger for signal and reference collection. And you need to tell the TimeTagger when to collect signal data and when to collect reference data. You can design the patterns of two digital output channels from the Digilent and send the signal to the TimeTagger.
- Use the 'CountBetweenMarkers' measurement class to collect your signal and reference data.

In [ ]:
# CountBetweenMarkers
n_rep = 1000000

# The default value is 0.5 V. Changing it to 0.4 V makes the 'CountBetweenMarkers' better.
tagger.setTriggerLevel(channel=3, voltage=0.40)     
tagger.setTriggerLevel(channel=4, voltage=0.40)

# Count the data from channel 1. Use channel 3 as the trigger. 3/-3 indicate the rising/falling edge of that channel
sig_counter = TimeTagger.CountBetweenMarkers(tagger=tagger, 
                                             click_channel=1, 
                                             begin_channel=3, 
                                             end_channel=-3, 
                                             n_values= n_rep)

ref_counter = TimeTagger.CountBetweenMarkers(tagger=tagger, 
                                            click_channel=1, 
                                            begin_channel=4, 
                                            end_channel=-4, 
                                            n_values= n_rep)

## Data analysis
- Fit your ODMR data to a Lorentzian function and obtain the transition frequency.
- Measure multiples NV centers. Do they have the same transition frequency or not?
- Move the magnet position. Check if the transition frequency changes as expected. Hint: when you are running your first experiment, you can remove the magnet or put it at the farthest position from the diamond, so that you know the expected transition frequency.

# Pulse experiment

## Pulse design

- Write a function or class to conveniently design your experiment pulse sequence. You should be able to modify the width of a certain part of your pulse sequence easily. For example, in your Rabi experiment you need to sweep the microwave pulse width.
- Connect the digital output channels to the oscilloscope to verify your pulse design. You can also use the photodiode to detect your green laser pulses. 


## Delay time
- When you design the pulse sequence, you are specifying the timing of the digital output channels from the Digilent. You use these digital output channels to control different devices, like the laser AOM, microwave switch and the timing in the TimeTagger. Those different devices have different response times. You need to compensate these different response times by adding delay times in your pulse sequence.

- The laser AOM especially has a long response time. Use the photodiode to measure the required delay time for the AOM.

## NV signal readout
- As you learned in the class, we can readout NV spin state by collecting the PL from the green excitation. It's important to collect only the initial 300 ns of the NV PL as the NV spin state will be polarized by the green laser afterwards. So you also need to be careful about the delay time when specifying your readout window.
- Like what you did in the ODMR experiment, you can design another readout window for your reference data.

## Data analysis
- Run a Rabi measurement to calibrate your $\pi$ and $\pi/2$ pulse times. For this experiment, as the duty cyle is relatively low, you can use high microwave power, but don't exceed the limit of the amplifier (9 dBm).
- Use your calibrated $\pi$ and $\pi/2$ pulses to run Ramsey and Hahn echo measurements.